In [1]:
import xarray as xr
import pandas as pd
import plotly.express as px

In [2]:
hot_ctd = xr.open_zarr("../../2_processed/hot_ctd.zarr")
hot_primary_production = xr.open_zarr("../../2_processed/hot_primary_production.zarr")
hot_zooplankton = xr.open_zarr("../../2_processed/hot_zooplankton.zarr")

In [3]:
hot_zooplankton

<xarray.Dataset> Size: 3MB
Dimensions:  (time: 1554, lat: 1, lon: 1, depth: 8, frac: 5)
Coordinates:
  * depth    (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac     (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
  * lat      (lat) float64 8B 22.75
  * lon      (lon) int64 8B -158
  * time     (time) datetime64[ns] 12kB 1994-02-17T11:08:00 ... 2022-09-02T22...
Data variables:
    abnd     (time, lat, lon, depth, frac) float64 497kB dask.array<chunksize=(777, 1, 1, 8, 5), meta=np.ndarray>
    carb     (time, lat, lon, depth, frac) float64 497kB dask.array<chunksize=(777, 1, 1, 8, 5), meta=np.ndarray>
    dwt      (time, lat, lon, depth, frac) float64 497kB dask.array<chunksize=(777, 1, 1, 8, 5), meta=np.ndarray>
    nit      (time, lat, lon, depth, frac) float64 497kB dask.array<chunksize=(777, 1, 1, 8, 5), meta=np.ndarray>
    svol     (time, lat, lon, depth, frac) float64 497kB dask.array<chunksize=(777, 1, 1, 8, 5), meta=np.ndarray>
    vol      (time, lat, lon, depth, frac) float64 497kB dask.array<chunksize=(777, 1, 1, 8, 5), meta=np.ndarray>
    wwt      (time, lat, lon, depth, frac) float64 497kB dask.array<chunksize=(777, 1, 1, 8, 5), meta=np.ndarray>

## Manage index

---


In [4]:
times = pd.DataFrame(
    {
        "btl_time": hot_ctd.time.to_series(),
        "zpk_time": hot_zooplankton.time.to_series(),
        "pp_time": hot_primary_production.time.to_series(),
    }
)
fig = px.histogram(
    times,
    x=["btl_time", "zpk_time", "pp_time"],
    title="time distribution",
    opacity=0.5,
    log_y=True,
    nbins=30,
    barmode="overlay",
    marginal="box",
    labels={"value": "time (m)", "variable": "Datasets"},
)

fig.show()

In [5]:
depths = pd.DataFrame(
    {
        "ctd_depth": hot_ctd.where(hot_ctd.depth < 400, drop=True).depth.to_series(),
        "zpk_depth": hot_zooplankton.depth.to_series(),
        "pp_depth": hot_primary_production.depth.to_series(),
    }
)
px.box(
    depths.stack().reset_index().rename(columns={"level_1": "dataset"}),
    x="dataset",
    y="depth",
)

## Migrant and resident


In [6]:
with xr.set_options(keep_attrs=True):
    night_condition = (hot_zooplankton.time.dt.hour > 18) | (
        hot_zooplankton.time.dt.hour < 6
    )

    hot_zooplankton = hot_zooplankton.assign_coords(is_night=night_condition)
    hot_zooplankton.is_night.attrs = {
        "flag_values": f"{[True, False]}",
        "flag_meanings": "night day",
        "standard_name": "is_night",
        "long_name": "Is night",
    }
    zpk_resident = (
        hot_zooplankton.swap_dims({"time": "is_night"})
        .sel(is_night=False)
        .swap_dims({"is_night": "time"})
        .drop_vars("is_night")
    )

    zpk_migrant = (
        hot_zooplankton.swap_dims({"time": "is_night"})
        .sel(is_night=True)
        .swap_dims({"is_night": "time"})
        .drop_vars("is_night")
    )
    zpk_resident = zpk_resident.resample(time="1D").mean().dropna("time", how="all")
    zpk_migrant = zpk_migrant.resample(time="1D").mean().dropna("time", how="all")
    zpk_migrant = zpk_migrant - zpk_resident
    zpk_migrant = xr.where(zpk_migrant > 0, zpk_migrant, 0).dropna("time", how="all")
    zpk_migrant = zpk_migrant[["carb", "dwt", "wwt"]].rename(
        {"carb": "carb_migrant", "dwt": "dwt_migrant", "wwt": "wwt_migrant"}
    )
    zpk_resident = zpk_resident[["carb", "dwt", "wwt"]].rename(
        {"carb": "carb_resident", "dwt": "dwt_resident", "wwt": "wwt_resident"}
    )
    hot_zooplankton = xr.merge([zpk_migrant, zpk_resident])
    # reorder dims as : time -> lat -> lon -> depth -> frac
    hot_zooplankton = hot_zooplankton.transpose("frac", "time", "lat", "lon", "depth")
hot_zooplankton

<xarray.Dataset> Size: 968kB
Dimensions:        (depth: 8, frac: 5, lat: 1, lon: 1, time: 502)
Coordinates:
  * depth          (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac           (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
  * lat            (lat) float64 8B 22.75
  * lon            (lon) int64 8B -158
  * time           (time) datetime64[ns] 4kB 1994-02-17 ... 2022-09-02
Data variables:
    carb_migrant   (frac, time, lat, lon, depth) float64 161kB dask.array<chunksize=(5, 1, 1, 1, 8), meta=np.ndarray>
    dwt_migrant    (frac, time, lat, lon, depth) float64 161kB dask.array<chunksize=(5, 1, 1, 1, 8), meta=np.ndarray>
    wwt_migrant    (frac, time, lat, lon, depth) float64 161kB dask.array<chunksize=(5, 1, 1, 1, 8), meta=np.ndarray>
    carb_resident  (frac, time, lat, lon, depth) float64 161kB dask.array<chunksize=(5, 1, 1, 1, 8), meta=np.ndarray>
    dwt_resident   (frac, time, lat, lon, depth) float64 161kB dask.array<chunksize=(5, 1, 1, 1, 8), meta=np.ndarray>
    wwt_resident   (frac, time, lat, lon, depth) float64 161kB dask.array<chunksize=(5, 1, 1, 1, 8), meta=np.ndarray>

## Gathering

---


# TODO : Select only the wanted variables in each dataset


In [10]:
final_dataset = xr.merge(
    [
        hot_zooplankton,
        hot_primary_production[
            [
                "chl",
            ]
        ],
        hot_ctd,
    ]
)
final_dataset

<xarray.Dataset> Size: 2MB
Dimensions:        (depth: 8, frac: 5, lat: 1, lon: 1, time: 659)
Coordinates:
  * depth          (depth) int64 64B 50 100 150 200 250 300 350 400
  * frac           (frac) float64 40B 0.2 0.5 1.0 2.0 5.0
  * lat            (lat) float64 8B 22.75
  * lon            (lon) int64 8B -158
  * time           (time) datetime64[ns] 5kB 1988-10-31 ... 2022-09-02
Data variables: (12/16)
    carb_migrant   (frac, time, lat, lon, depth) float64 211kB dask.array<chunksize=(5, 61, 1, 1, 8), meta=np.ndarray>
    dwt_migrant    (frac, time, lat, lon, depth) float64 211kB dask.array<chunksize=(5, 61, 1, 1, 8), meta=np.ndarray>
    wwt_migrant    (frac, time, lat, lon, depth) float64 211kB dask.array<chunksize=(5, 61, 1, 1, 8), meta=np.ndarray>
    carb_resident  (frac, time, lat, lon, depth) float64 211kB dask.array<chunksize=(5, 61, 1, 1, 8), meta=np.ndarray>
    dwt_resident   (frac, time, lat, lon, depth) float64 211kB dask.array<chunksize=(5, 61, 1, 1, 8), meta=np.ndarray>
    wwt_resident   (frac, time, lat, lon, depth) float64 211kB dask.array<chunksize=(5, 61, 1, 1, 8), meta=np.ndarray>
    ...             ...
    press          (time, lat, lon, depth) float64 42kB dask.array<chunksize=(659, 1, 1, 8), meta=np.ndarray>
    sal            (time, lat, lon, depth) float64 42kB dask.array<chunksize=(659, 1, 1, 8), meta=np.ndarray>
    sigma          (time, lat, lon, depth) float64 42kB dask.array<chunksize=(659, 1, 1, 8), meta=np.ndarray>
    temp           (time, lat, lon, depth) float64 42kB dask.array<chunksize=(659, 1, 1, 8), meta=np.ndarray>
    theta          (time, lat, lon, depth) float64 42kB dask.array<chunksize=(659, 1, 1, 8), meta=np.ndarray>
    xmiss          (time, lat, lon, depth) float64 42kB dask.array<chunksize=(659, 1, 1, 8), meta=np.ndarray>